# Transfer Learning Example on Radar Dataset (Grand Canyon)

## Importing the required modules

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import scipy.io as sio
import pandas as pd
import os
import re
import math
import itertools
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from itertools import product
from sklearn.utils.class_weight import compute_class_weight
torch.cuda.empty_cache()

# Reading the labels and creating train and test data

In [2]:
def index(string):
    s = re.findall("[0-9]", string)
    return int(''.join(s))

scenario_idx = 29 # Bonneville Salt Flats
names = os.listdir(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names = sorted(names, key=index)
print(len(names))
y = pd.read_csv(f'num{scenario_idx}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names = y.columns[4:7]
y = y[col_names].to_numpy()

scenario_idx_test = 35 # Grand Canyon
names_test = os.listdir(f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names_test = sorted(names_test, key=index)
y_test = pd.read_csv(f'num{scenario_idx_test}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names_test = y_test.columns[4:7]
y_test = y_test[col_names_test].to_numpy()

# Define fine-tuning name and labels
few_shot_count = 64
names_FS = names_test[:few_shot_count]
y_FS = y_test[:few_shot_count]

# Create training and testing datasets
y_train = y[:int(0.9 * len(names))]
y_test = y_test[int(0.9 * len(names_test)) - 1:]
training_names = names[:int(0.9 * len(names))]
test_names = names_test[int(0.9 * len(names_test)) - 1:]

print('Training labels:')
print(y_train)

# Tensor Corners
##################################################################################
# num29: [10851, 215, -5.4], num60: [11073, 215, -5.3], num62: [11471, 215, -5.6]
# num76: [11388, 215, -6.15], num35: [11381, 215, -0.95]
##################################################################################

# Training dataset global constants
coord_tr = [10851, 215, -5.4]   # Tensor corner
rng_res_tr = 59.9585 / 2        # Range resolution
az_step_tr = 0.4                # Azimuth step size
el_step_tr = 0.01               # Elevation step size

# Test dataset global constants
coord_ts = [11381, 215, -0.95]  # Tensor corner
rng_res_ts = 59.9585 / 2        # Range resolution
az_step_ts = 0.4                # Azimuth step size
el_step_ts = 0.01               # Elevation step size

def Drawing_Batch(names, label, bs, ind, directory, normalize=True):
    x = []
    labels = []
    
    for j in range(ind * bs, (ind + 1) * bs):
        try:
            temp = sio.loadmat(directory + names[j])['P']
        except:
            break
        if normalize:
            Anorm = temp - np.min(temp.flatten())
            temp = np.divide(Anorm, np.max(Anorm.flatten()))
        x.append(temp)
        labels.append(label[j, :])
        
    x = torch.FloatTensor(np.array(x))
    labels = torch.FloatTensor(np.array(labels))
    return x, labels

100000
Training labels:
[[ 8.3355 13.325  19.525 ]
 [18.156  17.388  32.775 ]
 [ 8.777  24.73   18.582 ]
 ...
 [15.785  15.992  29.641 ]
 [ 3.1736 10.716  12.27  ]
 [ 4.5552 21.035  14.234 ]]


In [3]:
print(len(y_train))
print(len(y_test))

90000
10001


## Define a regression CNN and instantiating it

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(21, 32, 3, 1)
        self.conv2 = nn.Conv1d(32, 64, 3, 1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64 * 5, 20)  # Adjust input size based on the output of conv layers and max pooling
        self.fc2_reg = nn.Linear(20, 2)  # Adjusted output size

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = F.max_pool1d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool1d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output_reg = self.fc2_reg(x)  # (bs, 2)
        
        return output_reg
    
from torchsummary import summary
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = Net()
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
print(summary(model,(21,26)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 24]           2,048
       BatchNorm1d-2               [-1, 32, 24]              64
            Conv1d-3               [-1, 64, 10]           6,208
       BatchNorm1d-4               [-1, 64, 10]             128
            Linear-5                   [-1, 20]           6,420
            Linear-6                    [-1, 2]              42
               Net-7                    [-1, 2]               0
Total params: 14,910
Trainable params: 14,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.06
Estimated Total Size (MB): 0.08
----------------------------------------------------------------
None


## Global Definitions

In [5]:
bs = 128  # batch_size
num_epoch = 10  # number of epochs
PATH = './ckpt_model.pth'   # forsaving the model
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Define a Loss function and optimizer; Using GPU or CPU
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    
def Spher2Cart_1D(spherical):
    cartesian = np.zeros(3)
    hypotenuse = np.cos(np.radians(spherical[2]))*spherical[0]
    cartesian[0] = np.cos(np.radians(spherical[1]))*hypotenuse
    cartesian[1] = -np.sin(np.radians(spherical[1]))*hypotenuse
    cartesian[2] = np.sin(np.radians(spherical[2]))*spherical[0]
    return cartesian

## Train and evaluate the network

In [6]:
def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

start = time.time()
main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True)
end = time.time()
print(end - start)

Epoch 0/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8219.14628559  7731.56993467 -1006.99025773]
[-8519.12322118  8442.27271505  -194.29731319]
[-9081.69542488  7213.07695345  -187.8823746 ]
Train Loss: 11.360616 ---- Test Loss: 86.396618
Epoch 1/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8194.42767714  7746.51625189 -1006.30048578]
[-8519.12322118  8442.27271505  -194.29731319]
[-8967.39812062  7305.75192641  -187.38063815]
Train Loss: 0.207086 ---- Test Loss: 69.206835
Epoch 2/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8190.37833086  7751.3793355  -1006.33617578]
[-8519.12322118  8442.27271505  -194.29731319]
[-8891.52041431  7318.53298694  -186.56078202]
Train Loss: 0.136451 ---- Test Loss: 68.116810
Epoch 3/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8189.49821671  7754.39110007 -1006.46390652]
[-8519.12322118  8442.27271505  -194.29731319]
[-8855.40051635  7322.97342986  -186.15520101]
Train Loss: 0.1124

In [7]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 7.547080267597325
Localization Error (m) = 641.2023282819082


## Fine-Tuning Regression CNN

In [8]:
model.module.conv1.weight.requires_grad = False
model.module.conv1.bias.requires_grad = False
model.module.batchnorm1.weight.requires_grad = False
model.module.batchnorm1.bias.requires_grad = False

model.module.conv2.weight.requires_grad = False
model.module.conv2.bias.requires_grad = False
model.module.batchnorm2.weight.requires_grad = False
model.module.batchnorm2.bias.requires_grad = False

In [9]:
bs = 64  # batch_size
num_epoch = 100
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

main(names_FS, test_names, bs, num_epoch, y_FS, y_test, normalize=True)

Epoch 0/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-8182.04147984  7381.26913786 -1034.22520792]
[-8239.59111431  8056.30503705  -178.33329667]
[-8786.21228455  7370.55097882  -177.47689676]
Train Loss: 22.160118 ---- Test Loss: 68.896415
Epoch 1/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-8169.29734079  7409.03395757 -1035.08621328]
[-8239.59111431  8056.30503705  -178.33329667]
[-8826.32401825  7335.56965905  -177.60641232]
Train Loss: 19.933058 ---- Test Loss: 71.555292
Epoch 2/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-8156.37613407  7436.94331186 -1035.95129817]
[-8239.59111431  8056.30503705  -178.33329667]
[-8858.64637498  7324.57792346  -177.88294614]
Train Loss: 17.933056 ---- Test Loss: 68.775848
Epoch 3/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-8143.28511797  7464.89749339 -1036.81479212]
[-8239.59111431  8056.30503705  -178.33329667]
[-8878.58386272  7341.68445265  -178.28942432]
Train Loss: 

[-8239.59111431  8056.30503705  -178.33329667]
[-8503.3591296   8050.17487103  -181.20915545]
Train Loss: 9.950361 ---- Test Loss: 25.463505
Epoch 33/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7978.40127445  7675.52899939 -1039.06733602]
[-8239.59111431  8056.30503705  -178.33329667]
[-8505.76228214  8051.94761267  -181.255023  ]
Train Loss: 9.916780 ---- Test Loss: 25.737388
Epoch 34/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7984.45294746  7671.96615494 -1039.24501306]
[-8239.59111431  8056.30503705  -178.33329667]
[-8508.15808866  8053.87468252  -181.30244999]
Train Loss: 9.891981 ---- Test Loss: 26.032691
Epoch 35/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7990.18768349  7669.02758938 -1039.4421874 ]
[-8239.59111431  8056.30503705  -178.33329667]
[-8510.46069993  8055.91554439  -181.35004023]
Train Loss: 9.876574 ---- Test Loss: 26.345038
Epoch 36/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7995.4825

[-8239.59111431  8056.30503705  -178.33329667]
[-8473.20788939  8112.07946096  -181.53177271]
Train Loss: 9.857388 ---- Test Loss: 28.643342
Epoch 65/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7984.18127337  7683.46378537 -1039.97459336]
[-8239.59111431  8056.30503705  -178.33329667]
[-8474.14608126  8111.98765618  -181.54127802]
Train Loss: 9.855421 ---- Test Loss: 28.703482
Epoch 66/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7985.24657343  7683.19718051 -1040.02928891]
[-8239.59111431  8056.30503705  -178.33329667]
[-8475.18959742  8111.72236708  -181.55010513]
Train Loss: 9.853657 ---- Test Loss: 28.754841
Epoch 67/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7986.40077138  7682.77905157 -1040.08014537]
[-8239.59111431  8056.30503705  -178.33329667]
[-8476.28613666  8111.30163011  -181.55786298]
Train Loss: 9.852221 ---- Test Loss: 28.795221
Epoch 68/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7987.5936

[-8239.59111431  8056.30503705  -178.33329667]
[-8477.58831423  8109.97831376  -181.558266  ]
Train Loss: 9.848927 ---- Test Loss: 28.757751
Epoch 97/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7989.77321754  7679.54277538 -1040.09776988]
[-8239.59111431  8056.30503705  -178.33329667]
[-8477.69561163  8109.806393    -181.55762674]
Train Loss: 9.848874 ---- Test Loss: 28.748720
Epoch 98/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7989.88190965  7679.36581465 -1040.09361555]
[-8239.59111431  8056.30503705  -178.33329667]
[-8477.83304996  8109.57942628  -181.55673575]
Train Loss: 9.848797 ---- Test Loss: 28.736510
Epoch 99/100
----------
[-8030.11680344  7623.28012548 -1039.18831649]
[-7990.01898224  7679.13528809 -1040.08789834]
[-8239.59111431  8056.30503705  -178.33329667]
[-8477.99271053  8109.31470243  -181.55568949]
Train Loss: 9.848711 ---- Test Loss: 28.722237


In [10]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 3.342892600922403
Localization Error (m) = 337.3033336528585
